# Pandas

* <a href="#grundlagen">Basics: DataFrames</a>
* <a href="#import">Importing Data</a>
* <a href="#transformation">Data transformation</a>
* <a href="#analysis">Operationens for data processing and analysis</a>
* <a href="#zeitreihen">time series</a>
* <a href="#aufgaben">Tasks</a>

Pandas is a library for the processing and analysis of structured data. It focuses on multidimensional fields or tables (so-called **DataFrames**). Pandas is based on the NumPy library and supports all data structures (fields) and operations defined there. In addition, there are various operations for importing data as well as for transformation and processing via SQL-like operations.

In [ ]:
import pandas as pd

Pandas essentially offers three central data structures:
* Series
* DataFrames
* Index

**Series** are one-dimensional fields, where the index can be either implicit (field index) or explicit (a key):

In [ ]:
s1 = pd.Series([0.5, 1.5, 0.33, 3.0, 2.1])
s1

In [ ]:
s2 = pd.Series([0.5, 1.5, 0.33, 3.0, 2.1], index=['a', 'b', 'c', 'd', 'e'])
s2

Individual elements are then accessed via the index:

In [ ]:
s1[2]

In [ ]:
s2['d']

The most important structure is **DataFrames**. In principle, these are tables consisting of columns that can be labeled, and rows - the data records that are identified by a key. A DataFrame can also be seen as a set of Series objects, each representing a column. As a result, a DataFrame can be constructed from multiple Series objects.

In [ ]:
counties = ['Ilm-Kreis', 'Gotha', 'Nordhausen']
inhabitants = pd.Series([108539, 135326, 84210], index=counties)
areas = pd.Series([844, 936, 714], index=counties) 
df = pd.DataFrame({'inhabitants': inhabitants, 'area': areas})
df

Elements within a DataFrame can be accessed in various ways. First of all, DataFrames are just two-dimensional arrays:

In [ ]:
df.values

A column can be accessed via its column name:

In [ ]:
df['area']

This can be used to simply add new columns:

In [ ]:
df['density'] = df['inhabitants'] / df['area']
df

Since a column is just an array, elements can be accessed via the index.

In [ ]:
df['inhabitants']['Ilm-Kreis']

<a id="import">&nbsp;</a>
## Importing Data

Usually, however, DataFrames are not created manually, but the data is loaded from files. For standard formats like CSV or JSON there are predefined functions like `read_csv`, which directly create a DataFrame:

In [ ]:
counties = pd.read_csv('/data/Thueringen-Kreise.csv', encoding='utf-8', sep=';')
counties

A similar funciton exists for reading JSON files:

In [ ]:
jdf = pd.read_json('/data/example.json')
jdf

The desired format can be given via the `orient` option.

In [ ]:
jdf = pd.read_json('/data/example.json', orient='index')
jdf

Finally, data can also be read from an SQL database. This can be achieved via the `read_sql_table` or `read_query` functions.

In [ ]:
# add new columns
counties['Name'] = counties['Kreisfreie Stadt + Landkreis + Land']
# use as index
counties.set_index('Name', inplace=True)
# take records 0..23: Attention copy()!
counties = counties[0:23].copy()
# Add new computed column
counties['density'] = counties['Bevölkerung-insgesamt'] / counties['Fläche'] 
# Drop unnecessary columns
counties.drop(columns=['Kreisfreie Stadt + Landkreis + Land'], inplace=True)
counties

Note: most operations can either be performed directly on the current DataFrame (`inplace=True`) or return a new DataFrame. 

In the above example the `copy()` is used at one place. Without this method a **SettingwithCopyWarning** occurs. Explanation: Some panda operations only provide views on the original DataFrame, i.e. no new DataFrame is created. Operations that change a DataFrame (like the above `[0:23]` with the assignment to `circles`) then lead to problems. Therefore either
* with `copy()` the DataFrame can be copied or
* the result is assigned to a new DataFrame, e.g. `districts2 = districts[0:23]`.

Furthermore, we want to derive another DataFrame, which contains only the cities without counties. Since the name is used as an index and not as a normal column, we have to access it via `counties.index`. We also want to use string methods, therefore `counties.index.str`.

In [ ]:
cities = counties[counties.index.str.contains('Stadt')]
cities

Alternatively, regular expressions could be used. They are defined in the `re` module.

In [ ]:
import re
p = re.compile('^Stadt')
cities = counties[counties.index.str.match(p)]
cities

<a id="analysis">&nbsp;</a>
## Operations for data processing and analysis

A variety of operations are available for further data processing and analysis. These include simple statistics about the method `describe()`.

In [ ]:
counties.describe()

We already know filters:

In [ ]:
counties[counties['Fläche'] > 1000]

Data in a DataFrame can also be transformed. In the following example, the population figures are given in thousands and therefore provide incorrect values when imported:

In [ ]:
cities2 = pd.read_csv('/data/Thueringen-Staedte.csv', encoding='utf-8', sep=';')
cities2

In this case, you can specify the data type of the column and perform an explicit conversion. We remove the thousand point from the string and convert the string into an `int` value.

In [ ]:
cities2 = pd.read_csv('/data/Thueringen-Staedte.csv', encoding='utf-8', sep=';', dtype={ 'Stadt': str, 'Landkreis': str, 'Einwohner': str })
cities2

In [ ]:
cities2['Einwohner'] = cities2['Einwohner'].str.replace('.', '').astype(float)
cities2

NaN stands for "Not a Number" - in database language: a NULL value. Such values can either be deleted (`dropna()`) or replaced by defined values (`fillna()`).

In [ ]:
cities2.fillna(0)

Pandas also supports a number of SQL-like operations such as associations and grouping/aggregation.

A join links two tables through identically named columns or the specified columns (`left_on`, `right_on`). Data records without partners in the other table are not transferred.

In [ ]:
kdf = pd.read_csv('/data/Thueringen-Kreise.csv', encoding='utf-8', sep=';')
sdf = pd.read_csv('/data/Thueringen-Staedte.csv', encoding='utf-8', sep=';')
kdf.merge(sdf, left_on="Kreisfreie Stadt + Landkreis + Land", right_on="Landkreis")

If, on the other hand, these are to appear in the result, an outer join must be executed.

In [ ]:
kdf.merge(sdf, left_on="Kreisfreie Stadt + Landkreis + Land", right_on="Landkreis",how="outer")

Another important operation is grouping (in SQL `group by`). In the DataFrame `cities2` the cities should be grouped according to their county and the population figures summed. For this we use the aggregate function `sum` from the NumPy module.

In [ ]:
import numpy as np
cities2.groupby('Landkreis').aggregate(np.sum)

Groups can also be filtered again, e.g. to remove all groups with 2 or less entries.

In [ ]:
cities2.groupby('Landkreis').filter(lambda x: x['Stadt'].count() > 2)

Note that a new grouping is necessary for an aggregation:

In [ ]:
cities2.groupby('Landkreis').filter(lambda x: x['Stadt'].count() > 2).groupby('Landkreis').aggregate(np.sum)

Besides this simple grouping, Pandas also supports the creation of pivot tables (cross tables) via `pivot_table` in order to represent multidimensional data sets in a two-dimensional table.

<a id="zeitreihen">&nbsp;</a>
##  Time series

Pandas was originally developed for the analysis of financial data. It therefore provides comprehensive support for time series. A time series is a series or DataFrame object indexed by time or date.

In [ ]:
idx = pd.DatetimeIndex(['2019-09-23','2019-09-24','2019-09-25','2019-09-26'])
data = pd.Series([10, 20, 15, 18], index=idx)
data

In [ ]:
# Using the date as index
data['2019-09-25':]

In addition to various functions for handling date and time values, functions for resampling, smoothing, etc. are also available. For demonstration purposes we upload the Apple share prices:

In [ ]:
from pandas_datareader import data
apple = data.DataReader('AAPL', 'yahoo',  start='2010', end='2019')
apple.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
apple = apple['Close']
apple.plot()

Resampling reduces the frequency - how can be specified by a frequency code. In the following example, we use the end of the fiscal year (`BA`).

In [ ]:
apple.plot(style='-')
apple.resample('BA').mean().plot(style='--')

<a id="aufgaben">&nbsp;</a>
##  Tasks

1. A CSV file is given with 3 fields per line with a comma as separator. The last field in each line is a string in the form `Value1|Value2|Value3`. Load the file into a DataFrame and transform the third field into three separate columns (value1 ... value3) of the DataFrame. Note: The Python `split` method can be used to split a string.